# RNN 4D - no collapsing
# V2.5 - run on lambda

In [1]:
import nibabel as nib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import numpy as np
import glob
import nibabel as nib
import os
import matplotlib.pyplot as plt
import scipy.ndimage
import random
from tensorflow.keras.layers import Dropout, Dense, Reshape, Flatten, Conv3D, Conv3DTranspose, LeakyReLU, Input, Embedding, multiply, Concatenate
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, RMSprop
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, BatchNormalization, Bidirectional, AdditiveAttention, LayerNormalization
from functools import partial
from tensorflow.keras import models, layers

In [2]:
#pip install nibabel

In [3]:
#pip install numpy==1.24.0

In [4]:
full_schizophrenia_ids = [
    'A00009280', 'A00028806', 'A00023132', 'A00014804', 'A00016859', 'A00021598', 'A00001181', 'A00023158',
    'A00024568', 'A00028405', 'A00001251', 'A00000456', 'A00015648', 'A00002405', 'A00027391', 'A00016720',
    'A00018434', 'A00016197', 'A00027119', 'A00006754', 'A00009656', 'A00038441', 'A00012767', 'A00034273',
    'A00028404', 'A00035485', 'A00024684', 'A00018979', 'A00027537', 'A00004507', 'A00001452', 'A00023246',
    'A00027410', 'A00014719', 'A00024510', 'A00000368', 'A00019293', 'A00014830', 'A00015201', 'A00018403',
    'A00037854', 'A00024198', 'A00001243', 'A00014590', 'A00002337', 'A00024953', 'A00037224', 'A00027616',
    'A00001856', 'A00037619', 'A00024228', 'A00038624', 'A00037034', 'A00037649', 'A00022500', 'A00013216',
    'A00020787', 'A00028410', 'A00002480', 'A00028303', 'A00020602', 'A00024959', 'A00018598', 'A00014636',
    'A00019349', 'A00017147', 'A00023590', 'A00023750', 'A00031597', 'A00015518', 'A00018317', 'A00016723',
    'A00021591', 'A00023243', 'A00017943', 'A00023366', 'A00014607', 'A00020414', 'A00035003', 'A00028805',
    'A00029486', 'A00000541', 'A00028408', 'A00000909', 'A00031186', 'A00000838' ]

# schizohrenia_id that satisfy t>90, 59 in total
met_requirement_schizophrenia_ids = [
    'A00000368', 'A00000456', 'A00000541', 'A00000838', 'A00001251', 'A00001452', 'A00004507',
    'A00006754', 'A00009280', 'A00012767', 'A00013216', 'A00014607', 'A00014719', 'A00014804',
    'A00014830', 'A00015201', 'A00015648', 'A00016197', 'A00016720', 'A00016723', 'A00017147',
    'A00018317', 'A00018403', 'A00018434', 'A00018979', 'A00019293', 'A00020414', 'A00020602', 
    'A00020787', 'A00021591', 'A00021598', 'A00023158', 'A00023246', 'A00023590', 'A00023750', 
    'A00024198', 'A00024228', 'A00024568', 'A00024684', 'A00024953', 'A00024959', 'A00027410', 
    'A00027537', 'A00028303', 'A00028404', 'A00028408', 'A00028805', 'A00028806', 'A00031186', 
    'A00031597', 'A00034273', 'A00035003', 'A00035485', 'A00037034', 'A00037224', 'A00037619', 
    'A00037649', 'A00038441', 'A00038624']

full_control_ids = [
    'A00007409', 'A00013140', 'A00021145', 'A00036049', 'A00022810', 'A00002198', 'A00020895', 'A00004667',
    'A00015826', 'A00023120', 'A00022837', 'A00010684', 'A00009946', 'A00037318', 'A00033214', 'A00022490',
    'A00023848', 'A00029452', 'A00037564', 'A00036555', 'A00023095', 'A00022729', 'A00024955', 'A00024160',
    'A00011725', 'A00027487', 'A00024446', 'A00014898', 'A00015759', 'A00028409', 'A00017294', 'A00014522',
    'A00012995', 'A00031764', 'A00025969', 'A00033147', 'A00018553', 'A00023143', 'A00036916', 'A00028052',
    'A00023337', 'A00023730', 'A00020805', 'A00020984', 'A00000300', 'A00010150', 'A00024932', 'A00035537',
    'A00022509', 'A00028406', 'A00004087', 'A00035751', 'A00023800', 'A00027787', 'A00022687', 'A00023866',
    'A00021085', 'A00022619', 'A00036897', 'A00019888', 'A00021058', 'A00022835', 'A00037495', 'A00026945',
    'A00018716', 'A00026907', 'A00023330', 'A00016199', 'A00037238', 'A00023131', 'A00014120', 'A00021072',
    'A00037665', 'A00022400', 'A00003150', 'A00024372', 'A00021081', 'A00022592', 'A00022653', 'A00013816',
    'A00014839', 'A00031478', 'A00014225', 'A00013363', 'A00037007', 'A00020968', 'A00024301', 'A00024820',
    'A00035469', 'A00029226', 'A00022915', 'A00022773', 'A00024663', 'A00036844', 'A00009207', 'A00024535',
    'A00022727', 'A00011265', 'A00024546'
]

 # 82 controls that met requirement
met_requirement_control_ids = [
    'A00000300', 'A00002198', 'A00003150', 'A00004087', 'A00007409', 'A00010684', 'A00011265', 'A00011725',
    'A00012995', 'A00013140', 'A00013816', 'A00014839', 'A00014898', 'A00015759', 'A00015826', 'A00018553',
    'A00018716', 'A00019888', 'A00020805', 'A00020895', 'A00020968', 'A00020984', 'A00021058', 'A00021072',
    'A00021081', 'A00021085', 'A00022400', 'A00022490', 'A00022509', 'A00022592', 'A00022619', 'A00022653',
    'A00022687', 'A00022727', 'A00022729', 'A00022773', 'A00022810', 'A00022835', 'A00022837', 'A00022915',
    'A00023095', 'A00023120', 'A00023131', 'A00023143', 'A00023330', 'A00023337', 'A00023730', 'A00023800',
    'A00023848', 'A00023866', 'A00024160', 'A00024301', 'A00024372', 'A00024446', 'A00024535', 'A00024546', 
    'A00024663', 'A00024820', 'A00024932', 'A00024955', 'A00025969', 'A00026945', 'A00027487', 'A00027787', 
    'A00028052', 'A00028406', 'A00028409', 'A00029226', 'A00029452', 'A00031478', 'A00031764', 'A00033214', 
    'A00035751', 'A00036049', 'A00036555', 'A00036844', 'A00037007', 'A00037238', 'A00037318', 'A00037495', 
    'A00037564', 'A00037665'
]

In [5]:
# import nibabel as nib
# import os
# import shutil

# # Directory containing your .nii.gz files
# directory_path = '4D/'
# file_pattern = '*.nii.gz'  # Adjust as needed for your file pattern

# # Directory to move corrupt files, create if doesn't exist
# corrupt_files_dir = os.path.join(directory_path, 'corrupt_files')
# os.makedirs(corrupt_files_dir, exist_ok=True)

# # List to store paths of corrupt files
# corrupt_files = []

# # Iterate over all files in the directory
# for root, _, files in os.walk(directory_path):
#     for file in files:
#         if file.endswith('.nii.gz'):
#             file_path = os.path.join(root, file)
#             try:
#                 # Attempt to load the file
#                 t1_img = nib.load(file_path)
#                 # Attempt to read the data to ensure it's not corrupt
#                 t1_data = t1_img.get_fdata()

#             except (EOFError, OSError, nib.filebasedimages.ImageFileError) as e:
#                 # Log the corrupt file and the error message
#                 print(f"Corrupt file detected: {file_path} | Error: {e}")
#                 corrupt_files.append(file_path)

#                 # Optionally, move the corrupt file to a separate directory
#                 shutil.move(file_path, os.path.join(corrupt_files_dir, file))
#                 continue

# # Output the list of corrupt files
# if corrupt_files:
#     print(f"\nTotal corrupt files found: {len(corrupt_files)}")
#     for corrupt_file in corrupt_files:
#         print(corrupt_file)
# else:
#     print("No corrupt files found.")


In [6]:
# GAN Training Data Selection
gan_train_ids_schiz = random.sample(met_requirement_schizophrenia_ids, 5)
gan_test_ids_schiz = [id for id in met_requirement_schizophrenia_ids if id not in gan_train_ids_schiz]

gan_train_ids_control = random.sample(met_requirement_control_ids, 5)
gan_test_ids_control = [id for id in met_requirement_control_ids if id not in gan_train_ids_control]
gan_test_ids_control = random.sample(gan_test_ids_control,9)

''' data training for classifier '''
''' just use the same train set as GAN above '''

# Classifier Test Data Selection
classifier_test_ids = gan_test_ids_schiz + gan_test_ids_control

''' File loading '''
# Specify the directory and file pattern
directory_path = '4D/'
file_pattern = 'A*_????_func_FL_FD_RPI_DSP_MCF_SS_SM_Nui_CS_InStandard.nii.gz'

# Construct the full path pattern
path_pattern = f'{directory_path}/{file_pattern}'

# Use glob to find all matching files
matching_files = glob.glob(path_pattern)

''' File loading for GAN Training and classifer '''
''' But this time we have 2 separate GANs, 1 train on schizoprenia and 1 train on control'''

#classifier_image_data = []
#classifier_labels = []  # 1 for schizophrenia, 0 for non-schizophrenia
gan_image_data_schiz = []
gan_image_data_control = []

for file_path in matching_files:
    filename = os.path.basename(file_path)
    file_id = filename.split('_')[0]
    
    if file_id in gan_train_ids_schiz:
        t1_img = nib.load(file_path)
        t1_data = t1_img.get_fdata()
        

        if t1_data.shape[3] < 90:
            continue

        #t1_data = np.sum(t1_data, axis=1)
        #print('shape of image: ', t1_data.shape)
        gan_image_data_schiz.append(t1_data)

    if file_id in gan_train_ids_control:
        t1_img = nib.load(file_path)
        t1_data = t1_img.get_fdata()

        if t1_data.shape[3] < 90:
            continue

        #t1_data = np.sum(t1_data, axis=1)
        gan_image_data_control.append(t1_data)


print(f"Total GAN control loaded: {len(gan_image_data_control)}")
print(f"Total GAN schiz loaded: {len(gan_image_data_schiz)}")



'''Determine the maximum time-dimension size '''
max_z_size_schiz = max(img.shape[3] for img in gan_image_data_schiz)
max_z_size_control = max(img.shape[3] for img in gan_image_data_control)
max_t_size = max(max_z_size_schiz,max_z_size_control)


# Normalize and pad the data
def normalize_and_pad(data, max_t):
    normalized = (data - np.min(data)) / (np.max(data) - np.min(data)) * 2 - 1
    padded = np.pad(normalized, ((0, 0), (0, 0), (0, 0), (0, max_t - data.shape[3])), mode='constant')
    return padded

padded_data_schiz = [normalize_and_pad(img, max_t_size) for img in gan_image_data_schiz]
padded_data_control = [normalize_and_pad(img, max_t_size) for img in gan_image_data_control]

padded_data_array_schiz = padded_data_schiz
padded_data_array_control = padded_data_control
print("shape after normalization and padding", padded_data_array_control[0].shape)

Total GAN control loaded: 5
Total GAN schiz loaded: 5
shape after normalization and padding (91, 109, 91, 146)


In [7]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("Enabled memory growth for GPUs.")
    except RuntimeError as e:
        print(e)

Enabled memory growth for GPUs.


In [8]:
import numpy as np
import tensorflow as tf

# Batch size
batch_size = 2

# Function to flatten 3D volumes for each time step (t, x*y*z)
def flatten_3d_data(data):
    return np.reshape(data, (data.shape[3], data.shape[0] * data.shape[1] * data.shape[2]))  # (t, x*y*z)

# Flatten each 3D volume for each time step
flattened_data_schiz = [flatten_3d_data(img) for img in padded_data_array_schiz]
flattened_data_control = [flatten_3d_data(img) for img in padded_data_array_control]
print("Shape after flattening:", flattened_data_schiz[0].shape)  # Check the shape after flattening

# Create labels
labels_schiz = np.ones(len(flattened_data_schiz))
labels_control = np.zeros(len(flattened_data_control))

# Combine images and labels
train_images = flattened_data_schiz + flattened_data_control
train_labels = np.concatenate((labels_schiz, labels_control), axis=0)

# Shuffle indices
indices = np.arange(len(train_images))
np.random.shuffle(indices)

# Shuffle data based on indices
train_images = [train_images[i] for i in indices]
train_labels = train_labels[indices]

# Define a generator function to yield data batches
def data_generator(images, labels, batch_size):
    for i in range(0, len(images), batch_size):
        batch_images = images[i:i + batch_size]
        batch_labels = labels[i:i + batch_size]
        yield np.array(batch_images), np.array(batch_labels)

# Create TensorFlow Dataset from the generator
train_dataset = tf.data.Dataset.from_generator(
    lambda: data_generator(train_images, train_labels, batch_size),
    output_signature=(
        tf.TensorSpec(shape=(None, max_t_size, 91 * 109 * 91), dtype=tf.float32),  # Adjust shape if needed
        tf.TensorSpec(shape=(None,), dtype=tf.float32)
    )
)

# Prefetch for performance improvement
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

# Debug: Test the generator
for images, labels in train_dataset.take(1):
    print(f"Batch image shape: {images.shape}")
    print(f"Batch labels shape: {labels.shape}")


Shape after flattening: (146, 902629)
Batch image shape: (2, 146, 902629)
Batch labels shape: (2,)


In [9]:
'''
STA-LSTM uses spatial attention to focus on important regions in each 3D volume 
and temporal attention to prioritize critical time steps. 
The dual attention mechanism is highly effective for data with both spatial and temporal components.
Directly learning spatial and temporal features.
'''

'\nSTA-LSTM uses spatial attention to focus on important regions in each 3D volume \nand temporal attention to prioritize critical time steps. \nThe dual attention mechanism is highly effective for data with both spatial and temporal components.\nDirectly learning spatial and temporal features.\n'

In [10]:
# Parallelize
# Check available GPUs
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"Available GPUs: {[gpu.name for gpu in gpus]}")
else:
    print("No GPUs detected.")

# Initialize MirroredStrategy to utilize all available GPUs
strategy = tf.distribute.MirroredStrategy()  # Automatically detects and uses all GPUs

# Wrap model creation and training code within the strategy scope
with strategy.scope():
    def build_rnn_model():
        # Define input shape: (time_steps, flattened_features)
        time_steps = max_t_size  # Number of time points
        features_per_volume = 91 * 109 * 91  # Flattened size of each 3D volume
        input_shape = (time_steps, features_per_volume)

        # Input layer
        inputs = layers.Input(shape=input_shape)
    
        # Reshape to prepare for spatial attention
        reshaped = layers.Reshape((time_steps, 91, 109, 91))(inputs)  # Reshape to (t, x, y, z)

        # Apply spatial attention: Flatten each 3D volume and use attention
        spatial_flatten = layers.TimeDistributed(layers.Flatten())(reshaped)
        spatial_attention = layers.MultiHeadAttention(num_heads=4, key_dim=64)(spatial_flatten, spatial_flatten)

        # Bidirectional LSTM layers to process the attended spatial features
        lstm_out = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(spatial_attention)

        # Apply temporal attention on the LSTM outputs
        temporal_attention = layers.MultiHeadAttention(num_heads=4, key_dim=128)(lstm_out, lstm_out)

        # Final LSTM layer to integrate attended features
        final_lstm_out = layers.LSTM(64)(temporal_attention)

        # Output layer for binary classification
        outputs = layers.Dense(1, activation='sigmoid')(final_lstm_out)

        # Compile model
        model = models.Model(inputs, outputs)
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
        return model
    
    
    rnn_model = build_rnn_model()
    rnn_model.summary()

    # Metrics for training
    train_loss = tf.keras.metrics.Mean(name='train_loss')
    train_accuracy = tf.keras.metrics.BinaryAccuracy(name='train_accuracy')

    # Define number of epochs
    epochs = 50

    # Define a step function for distributed training
    @tf.function
    def train_step(images, labels):
        with tf.GradientTape() as tape:
            predictions = rnn_model(images, training=True)
            loss = tf.keras.losses.binary_crossentropy(labels, tf.squeeze(predictions, axis=1))

        # Compute gradients
        gradients = tape.gradient(loss, rnn_model.trainable_variables)

        # Apply gradients across all replicas
        rnn_model.optimizer.apply_gradients(zip(gradients, rnn_model.trainable_variables))

        # Update metrics
        train_loss.update_state(loss)
        train_accuracy.update_state(labels, tf.squeeze(predictions, axis=1))

    # Training loop using the existing train_dataset
    for epoch in range(epochs):
        train_loss.reset_states()
        train_accuracy.reset_states()

        # Iterate through the training dataset using strategy.run()
        for images, labels in train_dataset:
            # Use strategy.run to ensure that each step runs in the correct replica context
            strategy.run(train_step, args=(images, labels))

        # Print the loss and accuracy for the current epoch
        print(f"Epoch {epoch + 1}, Loss: {train_loss.result().numpy()}, Training Accuracy: {train_accuracy.result().numpy()}")

Available GPUs: ['/physical_device:GPU:0', '/physical_device:GPU:1', '/physical_device:GPU:2', '/physical_device:GPU:3', '/physical_device:GPU:4', '/physical_device:GPU:5', '/physical_device:GPU:6', '/physical_device:GPU:7']
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 146, 902629)]        0         []                            
                          

In [ ]:
# Evaluation not working properly

In [22]:
def resize_image(image, new_shape):
    factors = (
        new_shape[0] / image.shape[0],
        new_shape[1] / image.shape[1],
        new_shape[2] / image.shape[2]
    )
    return scipy.ndimage.zoom(image, factors, order=1)  # order=1 is bilinear interpolation


test_image_data = []
test_labels = []

test_ids = classifier_test_ids  # List of IDs to filter test data

for file_path in matching_files:
    filename = os.path.basename(file_path)
    file_id = filename.split('_')[0]
    
    if file_id in test_ids:
        t1_img = nib.load(file_path)
        t1_data = t1_img.get_fdata()
        if t1_data.shape[3] < 90:
            continue
    
        # Normalize the processed image
        processed_image_normalized = (t1_data - np.min(t1_data)) / (np.max(t1_data) - np.min(t1_data)) * 2 - 1

        # Pad or truncate the time dimension to match the expected size (max_t_size)
        current_t_size = processed_image_normalized.shape[3]
        
        if current_t_size < max_t_size:
            # Pad to max_t_size time steps if the current time dimension is smaller
            pad_size = max_t_size - current_t_size
            #pad_size = max_t_size
            processed_image_padded = np.pad(processed_image_normalized, ((0, 0), (0, 0), (0, 0), (0, pad_size)), mode='constant')
        elif current_t_size > max_t_size:
            # Truncate to max_t_size time steps if the current time dimension is larger
            processed_image_padded = processed_image_normalized[:, :, :, :max_t_size]
        else:
            processed_image_padded = processed_image_normalized  # No padding needed

        # Check final shape before flattening
        #print(f"Processed image padded shape: {processed_image_padded.shape}")

        # Flatten each 3D volume (91, 109, 91) for each time step (t)
        flattened_shape = (processed_image_padded.shape[3], processed_image_padded.shape[0] * processed_image_padded.shape[1] * processed_image_padded.shape[2])
        processed_image_flattened = np.reshape(processed_image_padded, flattened_shape)  # (t, x*y*z)

        # Verify the shape after flattening
        #print(f"Flattened image shape: {processed_image_flattened.shape}")

        test_image_data.append(processed_image_flattened)
        
        label = 1 if file_id in met_requirement_schizophrenia_ids else 0
        test_labels.append(label)

# Convert to numpy arrays for easier handling in TensorFlow
test_images_array = np.array(test_image_data)
test_labels_array = np.array(test_labels)

# Check the final shape of test images
#print("Final shape of test_images_array:", test_images_array.shape)
#print("Final shape of test_labels_array:", test_labels_array.shape)

batch_size=1

# Create a TensorFlow dataset from the numpy arrays
test_dataset = tf.data.Dataset.from_tensor_slices((test_images_array, test_labels_array)).batch(batch_size)


2024-09-10 05:24:07.256265: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 61.86GiB (rounded to 66419052544)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-09-10 05:24:07.256659: W tensorflow/tsl/framework/bfc_allocator.cc:497] _________________________________________________****_____________________**************************


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

In [30]:
def resize_image(image, new_shape):
    factors = (
        new_shape[0] / image.shape[0],
        new_shape[1] / image.shape[1],
        new_shape[2] / image.shape[2]
    )
    return scipy.ndimage.zoom(image, factors, order=1)  # order=1 is bilinear interpolation


test_image_data = []
test_labels = []

test_ids = classifier_test_ids  # List of IDs to filter test data

# Test data processing with generator
def test_data_generator():
    for file_path in matching_files:
        filename = os.path.basename(file_path)
        file_id = filename.split('_')[0]
        
        if file_id in test_ids:
            t1_img = nib.load(file_path)
            t1_data = t1_img.get_fdata()
            
            if t1_data.shape[3] < 90:
                continue

            # Normalize the processed image
            processed_image_normalized = (t1_data - np.min(t1_data)) / (np.max(t1_data) - np.min(t1_data)) * 2 - 1

            # Pad or truncate the time dimension to match max_t_size
            current_t_size = processed_image_normalized.shape[3]
            if current_t_size < max_t_size:
                pad_size = max_t_size - current_t_size
                processed_image_padded = np.pad(processed_image_normalized, ((0, 0), (0, 0), (0, 0), (0, pad_size)), mode='constant')
            elif current_t_size > max_t_size:
                processed_image_padded = processed_image_normalized[:, :, :, :max_t_size]
            else:
                processed_image_padded = processed_image_normalized

            # Flatten each 3D volume
            flattened_shape = (processed_image_padded.shape[3], processed_image_padded.shape[0] * processed_image_padded.shape[1] * processed_image_padded.shape[2])
            processed_image_flattened = np.reshape(processed_image_padded, flattened_shape)  # (t, x*y*z)

            label = 1 if file_id in met_requirement_schizophrenia_ids else 0
            yield processed_image_flattened, label

# Define the output signature for TensorFlow dataset
output_signature = (
    tf.TensorSpec(shape=(max_t_size, 91 * 109 * 91), dtype=tf.float32),
    tf.TensorSpec(shape=(), dtype=tf.int32)
)

# Create the dataset from the generator
test_dataset = tf.data.Dataset.from_generator(
    test_data_generator,
    output_signature=output_signature
).batch(batch_size)

# Disable sharding for evaluation
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.OFF
test_dataset = test_dataset.with_options(options)

# Run evaluation on CPU to manage memory and avoid GPU-specific issues
@tf.function
def evaluate_on_cpu():
    with tf.device('/CPU:0'):
        # Disable CuDNN for LSTM layers if needed to avoid GPU memory issues
        for layer in rnn_model.layers:
            if isinstance(layer, tf.keras.layers.LSTM):
                layer._could_use_cudnn = False  # Forces TensorFlow to use CPU-friendly LSTM

        # Evaluate the model on the test dataset
        try:
            loss, accuracy = rnn_model.evaluate(test_dataset)
            print(f"Test loss: {loss}, Test accuracy: {accuracy}")
        except Exception as e:
            print(f"An error occurred during evaluation: {e}")

# Run the evaluation function
evaluate_on_cpu()

INFO:tensorflow:Error reported to Coordinator: Exception encountered when calling layer 'forward_lstm' (type LSTM).

{{function_node __wrapped__CudnnRNN_device_/job:localhost/replica:0/task:0/device:GPU:1}} CUDNN_STATUS_BAD_PARAM
in tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc(1694): 'cudnnSetTensorNdDescriptor( tensor_desc.get(), data_type, sizeof(dims) / sizeof(dims[0]), dims, strides)' [Op:CudnnRNN]

Call arguments received by layer 'forward_lstm' (type LSTM):
  • inputs=tf.Tensor(shape=(0, 146, 902629), dtype=float32)
  • mask=None
  • training=False
  • initial_state=None
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/tensorflow/python/training/coordinator.py", line 293, in stop_on_exception
    yield
  File "/usr/lib/python3/dist-packages/tensorflow/python/distribute/mirrored_run.py", line 387, in run
    self.main_result = self.main_fn(*self.main_args, **self.main_kwargs)
  File "/usr/lib/python3/dist-packages/tensorflow/python/autograph/im

2024-09-10 05:37:59.256729: W tensorflow/core/framework/op_kernel.cc:1828] OP_REQUIRES failed at cudnn_rnn_ops.cc:1769 : UNKNOWN: CUDNN_STATUS_BAD_PARAM
in tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc(1694): 'cudnnSetTensorNdDescriptor( tensor_desc.get(), data_type, sizeof(dims) / sizeof(dims[0]), dims, strides)'


In [ ]:
# Predict the probabilities
predictions = rnn_model.predict(test_dataset)
# Convert probabilities to class labels
predicted_labels = (predictions > 0.5).astype(int)

# Assuming test_labels_array contains your actual labels
actual_labels = test_labels_array

# Now you might want to compare these predicted_labels with the actual labels (test_labels)
# to compute the confusion matrix, classification report, etc.
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(actual_labels, predicted_labels.flatten()))
print(classification_report(actual_labels, predicted_labels.flatten()))

In [ ]:
for actual, predicted in zip(actual_labels, predicted_labels):
    print(f'Actual: {actual}, Predicted: {predicted}')